# IMPORT LIBRARIES

In [ ]:
print("[INFO]: Importing modules.")
# libraries for Keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten
from keras.optimizers import Adam

# another
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import os
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from datetime import datetime

# current time
time = datetime.now().strftime("%Y-%m-%d %Hh-%Mm-%Ss")
print("[INFO]: Finish importing modules.")

# LOAD DATA TO TRAIN

In [ ]:
# input image size
height = 32
width = 32
channels = 3
# class amount
classes = 43
# ratio to split Data Train into 2 part Train and Val: 80% to train and 20% to val.
ratio = 0.2

# variable
data = []
labels = []

# reading the input images and putting them into a numpy array
print("[INFO]: Loading data.")
for i in range(classes):
    path = "./input/train/{0}/".format(i)
    print("[INFO]: Loading data class {}.".format(i))
    class_ = os.listdir(path)
    for item in class_:
        try:
            image = cv2.imread(path + item)
            image_from_array = Image.fromarray(image, 'RGB')
            size_image = image_from_array.resize((height, width))
            data.append(np.array(size_image))
            labels.append(i)
        except Exception as e:
            print("[EXCEPTION ERROR]: " + e)
            
Cells = np.array(data)
labels = np.array(labels)
print("[INFO]: Finish loading data.")

# LOAD TEST DATASET

In [ ]:
# variable
data = []

# load Test data
print("[INFO]: Loading test set.")
y_test=pd.read_csv("./input/Test.csv")
_labels=y_test['Path'].values
y_test=y_test['ClassId'].values
for f in _labels:
    path = './input/test/' + f.replace('Test/', '')
    image = cv2.imread(path)
    image_from_array = Image.fromarray(image, 'RGB')
    size_image = image_from_array.resize((height, width))
    data.append(np.array(size_image))

X_test = np.array(data)
X_test = X_test.astype('float32')/255
print("[INFO]: Finish loading test set.")

# DEFINE TRAIN MODEL

In [ ]:
# constant
EPOCHS = 60
BATCH_SIZE = 200
# compilation of the model
adam = Adam(lr=0.002)
max_score = 0.0
data_score = []

def trainModel(X_train, X_val, y_train, y_val, X_test, y_test):
    global max_score
    global data_score
    
    ## Prepare Data
    X_train = X_train.astype('float32')/255 
    X_val = X_val.astype('float32')/255
    y_train = to_categorical(y_train, classes)
    y_val = to_categorical(y_val, classes)
    
    ## Create Model
    model = Sequential()
    # Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x6. Activation = RELU.
    model.add(Conv2D(filters=6, kernel_size=(5,5), strides=1, padding='valid', activation='relu', input_shape=X_train.shape[1:]))
    # Pooling. Input = 28x28x6. Output = 14x14x6.
    model.add(MaxPool2D(pool_size=(2, 2), strides=2, padding='valid'))
    # Layer 2: Convolutional. Output = 10x10x16. Activation = RELU.
    model.add(Conv2D(filters=16, kernel_size=(5,5), strides=1, padding='valid', activation='relu'))
    # Pooling. Input = 10x10x16. Output = 5x5x16.
    model.add(MaxPool2D(pool_size=(2, 2), strides=2, padding='valid'))
    # Flatten. Input = 5x5x16. Output = 400.
    model.add(Flatten())
    # Layer 3: Fully Connected. Input = 400. Output = 120. Activation = RELU.
    model.add(Dense(120, activation='relu'))
    # Layer 4: Fully Connected. Input = 120. Output = 84. Activation = RELU.
    model.add(Dense(84, activation='relu'))
    # Layer 5: Fully Connected. Input = 84. Output = 43.
    model.add(Dense(43, activation='softmax'))
    model.compile(
    loss='categorical_crossentropy', 
    optimizer=adam, 
    metrics=['accuracy']
    )
    
    ## Train Model
    history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(X_val, y_val))
    
    ## Predict
    pred = model.predict_classes(X_test)
    score = accuracy_score(y_test, pred)
    data_score.append(score)
    print("[INFO]: Finish. Test Accuracy: {0}".format(score))
    
    if score > max_score:
        max_score = score
        ## Save Model
        save_path = "./model/{}_model{}.h5".format(time, str(EPOCHS))
        model.save(save_path)
        print("[INFO]: Saved model to " + save_path)

# TRAIN MODEL

In [ ]:
s = np.arange(Cells.shape[0])
np.random.seed(classes)
np.random.shuffle(s)

cv = KFold(n_splits=5, shuffle=True, random_state=np.random)
count = 0
for train_index, test_index in cv.split(Cells):
    count += 1
    X_train, X_val, y_train, y_val = Cells[train_index], Cells[test_index], labels[train_index], labels[test_index]

    print("[INFO]: Training model part {0}: ".format(count))
    trainModel(X_train, X_val, y_train, y_val, X_test, y_test)

# SUMMARY

In [ ]:
print("[INFO]: SUMMARY:")
for i in range(len(data_score)):
    print("[INFO]: Model {} accuracy: {}".format(str(i + 1), data_score[i]))
avg_accuracy = sum(data_score)/len(data_score)
print("-"*20)
print("[INFO]: Average accuracy: {}".format(avg_accuracy))
max_accuracy = max(data_score)
print("[INFO]: Highest accuracy: {}".format(max_accuracy))
y_pos = np.arange(1, len(data_score) + 1)
plt.bar(y_pos, data_score, color=(0.2, 0.4, 0.6, 0.6))
 
# Custom Axis title
plt.xlabel('Accuracy', fontweight='bold', color = 'orange', fontsize='17', horizontalalignment='center')